### 클래스 선언
- data_path: dataset이 위치한 폴더명  
- model: 모델이 저장된 폴더 또는 hugging face의 모델 uid  
  - BiEncoder는 SentenceTransformer로 불러올 수 있는 모델들  
  - CrossEncoder_는 AutoModel로 불러올 수 있는 모델들

In [1]:
from sbert import BiEncoder, CrossEncoder_

DATA_PATH = ""
MODEL_PATH = ""
retriever = BiEncoder(data_path=DATA_PATH, model=MODEL_PATH)
reranker = CrossEncoder_(data_path=DATA_PATH, model=MODEL_PATH)

### BiEncoder 클래스의 함수 설명
**`train`**  
- output_dir: 학습한 모델을 저장할 폴더의 경로(없으면 생성해줌) default: `./sbert-bi-encoder`  
- epoch: default 5
- batch_size: defualt 32
- lr: default 2e-5
- weight_decay: default 0.01

**`validate`**  
- data_type: `train`, `valid` 중 검증할 대상
- topk 개수를 설정하고 싶다면 [sbert.py](./sbert.py)에서 수정하기  

**`retrieve`**  
- data_type: `train`, `valid`, `test` 중 retrieval을 수행할 대상
- stage: retrieval만 수행하려면 1(DataFrame 반환), re-rank로 연결하려면 2(document_id와 score의 dict 반환)
- topk: query 당 상위 몇 개의 문서를 반환할 것인지

**`save_corpus_embeddings`**  
- save_path: corpus의 embedding vector를 pt 파일로 저장할 경로

In [ ]:
retriever.train()
retriever.validate('valid')
retriever.retrieve(data_type='test', stage=1, topk=5)
retriever.save_corpus_embeddings('./')

### CrossEncoder_ 함수 설명
> 기존의 CrossEncoder 클래스 이름과 겹쳐 `_` 추가  

**`train`**  
- output_dir: 학습한 모델을 저장할 폴더의 경로(없으면 생성해줌) default: `./sbert-cross-encoder`  
- epoch: default 5
- batch_size: defualt 32
- weight_decay: default 0.01
- lr은 CrossEncoder 클래스에서 초기화를 지원하지 않는 듯함

**`validate`**  
- data_type: `train`, `valid` 중 검증할 대상
- topk: query 당 상위 몇 개의 문서를 반환할 것인지  
CrossEncoder 클래스의 rank 함수를 사용하는데, 전체 corpus를 대상으로 하기 때문에 아주 오래 걸림(시도하지 않는 것 추천)  

**`retrieve`**  
CrossEncoder_만으로 retrieval을 수행할 때 사용하는 함수로, DataFrame을 반환  
- data_type: `train`, `valid`, `test` 중 retrieval을 수행할 대상
- topk: query 당 상위 몇 개의 문서를 반환할 것인지

**`rerank`**  
BiEncoder에서 얻은 결과로 re-ranking을 수행할 때 사용하는 함수로, DataFrame을 반환  
- hits: BiEncoder에서 `retrieve(stage=2)`를 수행하고 얻은 결과  
- data_type: `train`, `valid`, `test` 중 reranking을 수행할 대상으로, `retrieve(data_type='')`에서 사용한 값과 일치해야 함  
- topk: query 당 상위 몇 개의 문서를 반환할 것인지를 결정하며, 일반적으로 retrieval에서 100, reranking에서 10으로 설정

**`save_corpus_embeddings`**  
corpus의 embedding vector를 저장하고자 했으나, CrossEncoder에서 지원하는 기능이 아닌 듯하여 추후 방법을 찾으면 업데이트 예정

In [ ]:
reranker.train()
reranker.validate('valid')
reranker.retrieve(data_type='valid', topk=5)

# 2-stages
hits = retriever.retrieve(data_type='valid', stage=2, topk=5)
reranker.rerank(hits, data_type='valid', topk=3)